In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor
import yfinance as yf

## Functions 

In [20]:
def Pagan_test(data): 
    # Create DataFrame from data dictionary
    df = pd.DataFrame(data)

    # Drop rows with missing values to avoid errors during modeling
    df = df.dropna()

    # Build the formula string dynamically from your independent variables
    independent_vars = "+".join([f"X{i}" for i in range(1, 7)])
    formula = f"Y ~ {independent_vars}"

    # Fit the OLS regression model
    model = smf.ols(formula=formula, data=df).fit()

    # Run the Breusch-Pagan test
    bp_test = sms.het_breuschpagan(model.resid, model.model.exog)
    labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']

    # Print Breusch-Pagan test results
    for label, value in zip(labels, bp_test):
        print(f"{label}: {value}")

def Whites_test(data):
    df = pd.DataFrame(data).dropna()

    X = df[[f'X{i}' for i in range(1, 7)]]
    X = sm.add_constant(X)
    y = df['Y']

    model = sm.OLS(y, X).fit()

    white_test = sms.het_white(model.resid, model.model.exog)

    labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
    for label, value in zip(labels, white_test):
        print(f"{label}: {value}")

def p_valueTest(data):

    df = pd.DataFrame(data).dropna()

    # Assume df is your DataFrame with independent variables X1, X2,... and dependent variable Y
    X = df.drop('Y', axis=1)
    y = df['Y']

    # Add a constant to the model (intercept)
    X = sm.add_constant(X)

    # Fit the model
    model = sm.OLS(y, X).fit()

    # Get summary which includes p-values
    print(model.summary())

    # Or just get the p-values directly:
    p_values = model.pvalues
    print(p_values)

def LRM(data): 
    df = pd.DataFrame(data)

    df = std_filter(df)  # remove rows with missing data

    X = df.drop('Y', axis=1)
    y = df['Y']

    # Initialize and fit the model
    model = LinearRegression()
    model.fit(X, y)

    Coefs = []

    # Print coefficients to show the importance of each variable
    for var, coef in zip(X.columns, model.coef_):
        Coefs.append([var, coef])

    return Coefs

def VIF_test(data):
    df = pd.DataFrame(data).dropna()

    # Exclude dependent variable Y for VIF calculation
    X = df[[f"X{i}" for i in range(1, 7)]]

    # Add constant term for intercept
    X = sm.add_constant(X)

    # Calculate VIF for each explanatory variable
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    print(vif_data)

def std_filter(data):
    means = data.mean()
    stds = data.std()    

    filtered_values = []

    for idx in range(len(data)):
        row = data.iloc[idx]

        # Check if every column's value is within mean ± std specifically for that column
        in_range = ((row >= (means - stds)) & (row <= (means + stds))).all()

        if in_range:
            filtered_values.append(row)

    df_filtered = pd.DataFrame(filtered_values, columns=data.columns)
    return df_filtered

def plot_data_log(data): 
    df = pd.DataFrame(data)

    y = df.index

    for i in data: 
        plt.plot(df[i],y, label=i, marker='o', markersize=1, linestyle='')

    plt.xscale('log')

    plt.plot(df[i],y, label="nvda", marker='o', markersize=1, linestyle='')

    plt.legend()
    plt.xlabel('Independent Variables (Log Scale)')
    plt.ylabel('NVIDIA Stock Price (Y)')
    plt.title('Stock Prices vs NVIDIA Stock Price')

def plot_data_log_y(data, dotsize=1): 
    df = pd.DataFrame(data)

    for i in data: 

        if i == "Y":
            continue

        plt.plot(df[i], df["Y"], label=i, marker='o', markersize=dotsize, linestyle='')

    plt.xscale('log')

    # plt.plot(y,df["Y"], label="nvda", marker='o', markersize=1, linestyle='')

    plt.legend()
    plt.xlabel('Independent Variables (Log Scale)')
    plt.ylabel('NVIDIA Stock Price (Y)')
    plt.title('Stock Prices vs NVIDIA Stock Price')

def plot_data_log_SAVE(data): 
    df = pd.DataFrame(data)

    y = df.index

    for i in data: 
        plt.plot(df[i], y, label=i)

    plt.xscale('log')

    plt.plot(df["Y"], y, label="nvda")

    plt.legend()

    plt.xlabel('Independent Variables (Log Scale)')
    plt.ylabel('NVIDIA Stock Price (Y)')
    plt.title('Stock Prices vs NVIDIA Stock Price')

    plt.savefig('my_high_res_plot.png', dpi=300)

def Set_index(data, index):
    for i in data: 
        i.index = index

## Stocks import

In [4]:
nvda = yf.Ticker("NVDA") #
asml = yf.Ticker("ASML") #
aapl = yf.Ticker("AAPL") #
goog = yf.Ticker("GOOG") #
tsmc = yf.Ticker("TSM") #
s_p = yf.Ticker("^GSPC")
ndx = yf.Ticker("^NDX")
spgtsi = yf.Ticker("^SPGSTI")
amd = yf.Ticker("AMD")
intel = yf.Ticker("INTL")
copper = yf.Ticker("HG=F")
gold = yf.Ticker("GC=F")
silver = yf.Ticker("SI=F")
platinum = yf.Ticker("PL=F")
uranium = yf.Ticker("URA")
coal = yf.Ticker("COAL")
qcom = yf.Ticker("QCOM")

### MODEL 1: Related Stocks

In [5]:
data1 = {
    "Y": nvda.history(start="2020-01-01")['Close'],
    "X1": asml.history(start="2020-01-01")['Close'],
    "X2": aapl.history(start="2020-01-01")['Close'],
    "X3": goog.history(start="2020-01-01")['Close'],
    "X4": tsmc.history(start="2020-01-01")['Close'],
    "X5": intel.history(start="2020-01-01")['Close'],
    "X6": qcom.history(start="2020-01-01")['Close']
}

In [ ]:
LRM1 = LRM(data1)

for i in LRM1:
    print(i)    

['X1', -0.06181235233356901]
['X2', 0.17134894165325587]
['X3', 0.47146679656068946]
['X4', 0.7631286303881321]
['X5', -0.842583757637524]
['X6', -0.058808869382152996]


In [12]:
p_valueTest(data1)

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.972
Model:                            OLS   Adj. R-squared:                  0.971
Method:                 Least Squares   F-statistic:                     4148.
Date:                Fri, 07 Nov 2025   Prob (F-statistic):               0.00
Time:                        12:07:35   Log-Likelihood:                -2645.2
No. Observations:                 735   AIC:                             5304.
Df Residuals:                     728   BIC:                             5337.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -114.2241      8.357    -13.669      0.0

In [13]:
Pagan_test(data1)

Lagrange multiplier statistic: 105.04151360579209
p-value: 2.221729691030682e-20
f-value: 20.23155058537301
f p-value: 5.889942894730849e-22


In [14]:
Whites_test(data1)

Test Statistic: 258.2012968192203
Test Statistic p-value: 1.3477459129761334e-39
F-Statistic: 14.180090522819176
F-Test p-value: 4.295112728475618e-50


In [15]:
VIF_test(data1)

  Variable         VIF
0    const  649.740431
1       X1    3.837932
2       X2    5.650575
3       X3   12.918060
4       X4   21.159106
5       X5   11.574820
6       X6    4.807809


### MODEL 2: yearly metal production 

Importing data

In [16]:
silver_data = pd.read_excel('./Data/Silver Historical Statistics.xlsx', sheet_name="Silver")
Tin_data = pd.read_excel('./Data/Tin Historical Statistics.xlsx', sheet_name="Tin")
Silicon_data = pd.read_excel('./Data/Silicon Historical Statistics.xlsx', sheet_name="Silicon")
Aluminum_data = pd.read_excel('./Data/Aluminum Historical Statistics.xlsx', sheet_name="Aluminum")
Rare_Earths_data = pd.read_excel('./Data/Rare Earths Historical Statistics.xlsx', sheet_name="Rare earths")
Gold_data = pd.read_excel('./Data/Gold Historical Statistics.xlsx', sheet_name="Gold")

filtered_silver_data = silver_data[(124-20):124]["Unnamed: 11"]
filtered_Tin_data = Tin_data[(124-20):124]["Unnamed: 11"] 
filtered_Silicon_data = Silicon_data[(102-21):101]["Unnamed: 9"] 
filtered_Aluminum_data = Aluminum_data[(124-20):124]["Unnamed: 15"] 
filtered_Rare_Earths_data = Rare_Earths_data[(124-20):124]["Unnamed: 7"] 
filtered_Gold_data = Gold_data[(124-20):124]["Unnamed: 8"]

filtered_Silicon_data

81     3500000
82     3500000
83     3720000
84     4390000
85     4900000
86     5310000
87     5650000
88     6330000
89     6510000
90     6260000
91     6930000
92     7370000
93     7450000
94     7950000
95     7900000
96     7800000
97     7700000
98     7230000
99     8890000
100    8560000
Name: Unnamed: 9, dtype: object

In [47]:
data2 = {
    "Y": nvda.history(start="2000-01-01", end="2020-1-1")['Close'].resample('YE').last(),
    "X1": filtered_silver_data.reset_index(drop=True),
    "X2": filtered_Tin_data.reset_index(drop=True),
    "X3": filtered_Silicon_data.reset_index(drop=True),
    "X4": filtered_Aluminum_data.reset_index(drop=True),
    "X5": filtered_Rare_Earths_data.reset_index(drop=True),
    "X6": filtered_Gold_data.reset_index(drop=True)
}

In [41]:
filtered_Gold_data.index = nvda.history(start="2000-01-01", end="2020-1-1")['Close'].resample('YE').last().index
filtered_silver_data.index = nvda.history(start="2000-01-01", end="2020-1-1")['Close'].resample('YE').last().index
filtered_Tin_data.index = nvda.history(start="2000-01-01", end="2020-1-1")['Close'].resample('YE').last().index
filtered_Silicon_data.index = nvda.history(start="2000-01-01", end="2020-1-1")['Close'].resample('YE').last().index
filtered_Aluminum_data.index = nvda.history(start="2000-01-01", end="2020-1-1")['Close'].resample('YE').last().index
filtered_Rare_Earths_data.index = nvda.history(start="2000-01-01", end="2020-1-1")['Close'].resample('YE').last().index

In [43]:
LRM2 = LRM(data2)

for i in LRM2:
    print(i)

['X1', 1.3779569741740414e-07]
['X2', -1.0403748454425543e-06]
['X3', 2.454657470015021e-07]
['X4', -2.993600615202989e-08]
['X5', 2.818485953882873e-06]
['X6', 4.194505846170544e-09]


In [44]:
Pagan_test(data2)

Lagrange multiplier statistic: 20.0
p-value: 1.0
f-value: nan
f p-value: nan


In [45]:
Whites_test(data2)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [59]:
VIF_test(data2)

/var/folders/wb/pmxl4r0s6svgkb08w2drl86c0000gn/T/ipykernel_56338/25042352.py:80: RuntimeWarning: '<' not supported between instances of 'int' and 'Timestamp', sort order is undefined for incomparable objects.
  df = pd.DataFrame(data).dropna()
/var/folders/wb/pmxl4r0s6svgkb08w2drl86c0000gn/T/ipykernel_56338/25042352.py:80: RuntimeWarning: '<' not supported between instances of 'int' and 'Timestamp', sort order is undefined for incomparable objects.
  df = pd.DataFrame(data).dropna()
/var/folders/wb/pmxl4r0s6svgkb08w2drl86c0000gn/T/ipykernel_56338/25042352.py:80: RuntimeWarning: '<' not supported between instances of 'int' and 'Timestamp', sort order is undefined for incomparable objects.
  df = pd.DataFrame(data).dropna()
/var/folders/wb/pmxl4r0s6svgkb08w2drl86c0000gn/T/ipykernel_56338/25042352.py:80: RuntimeWarning: '<' not supported between instances of 'int' and 'Timestamp', sort order is undefined for incomparable objects.
  df = pd.DataFrame(data).dropna()
/var/folders/wb/pmxl4r0s

ValueError: zero-size array to reduction operation maximum which has no identity